In [ ]:
using Newtonsoft.Json;
var origamiInput = System.IO.File.ReadAllLines("input.txt");

In [ ]:
var grid = new Dictionary<int,List<int>>();

foreach(var coordinates in origamiInput.TakeWhile(x => !string.IsNullOrEmpty(x))){
    var c = coordinates.Split(",").Select(x => int.Parse(x)).ToArray();
    if(grid.ContainsKey(c[1])){
        var row = grid[c[1]];
        if(row.Any(x => x == c[0])){
            continue;
        }
        row.Add(c[0]);
    }else{
        grid.Add(c[1],new List<int>{ c[0] });
    }
}

var FoldInstructions = origamiInput
    .Reverse()
    .TakeWhile(x => x.StartsWith("fold along"))
    .Reverse()
    .Select(x => new { 
        fold = x[11] == 'y' ? Fold.Up : Fold.Left,
        position = int.Parse(x.Split("=")[1])
    }).ToList();
FoldInstructions


index,fold,position
0,Left,655
1,Up,447
2,Left,327
3,Up,223
4,Left,163
5,Up,111
6,Left,81
7,Up,55
8,Left,40
9,Up,27


In [ ]:
var length = grid.Max(x => x.Value.Max());
var height = grid.Max(x => x.Key);

public void DisplayGrid(){
    for(var x = 0; x <= height; x++){
        var cellQueue = grid.ContainsKey(x) 
            ? new Queue<int>(grid[x].OrderBy(c => c))
            : new Queue<int>();
        for(var y = 0; y <= length; y++){
            var isFilled = cellQueue.Count() > 0 ? cellQueue.Peek() == y : false;
            if(isFilled){
                cellQueue.Dequeue();
                Console.Write("#");
            }else{
                Console.Write(".");
            }
        }
        Console.WriteLine();
    }
}

In [ ]:
public enum Fold{
    Up,
    Left
}

public void FoldGrid(Fold direction, int position){

    if(direction == Fold.Up){
        for(var y = position + 1; y <= height; y++){
            var a = grid.ContainsKey(y) ? grid[y] : null;
            var bRow = Math.Max(position - (y - position), 0);
            var b = grid.ContainsKey(bRow) ? grid[bRow] : new List<int>();
    
            var r = b.Union(a ?? new List<int>()).Distinct().ToList();
            if(r.Any()){
                grid[bRow] = r;
            }
    
            if(a != null){
                grid.Remove(y);
            }
        }
    
        height = position - 1;
    }else{
        for(var x = position + 1; x <= length; x++){
            foreach(var row in grid){
                var a = row.Value.Any(c => c == x);
                var bColumn = Math.Max(position - (x - position), 0);
                var b = row.Value.Any(c => c == bColumn);

                var r = a || b;
                if(r && !b) row.Value.Add(bColumn);

                if(a) row.Value.Remove(x);
            }
        }

        length = position -1;
    }
}


In [ ]:
FoldInstructions.ForEach(f => {
    FoldGrid(f.fold, f.position);
    Console.WriteLine(JsonConvert.SerializeObject(new {
        fold = f,
        count = grid.Sum(x => x.Value.Count())
    }));
});

{"fold":{"fold":1,"position":655},"count":720}
{"fold":{"fold":0,"position":447},"count":594}
{"fold":{"fold":1,"position":327},"count":510}
{"fold":{"fold":0,"position":223},"count":417}
{"fold":{"fold":1,"position":163},"count":354}
{"fold":{"fold":0,"position":111},"count":281}
{"fold":{"fold":1,"position":81},"count":234}
{"fold":{"fold":0,"position":55},"count":201}
{"fold":{"fold":1,"position":40},"count":167}
{"fold":{"fold":0,"position":27},"count":145}
{"fold":{"fold":0,"position":13},"count":122}
{"fold":{"fold":0,"position":6},"count":104}


In [ ]:
DisplayGrid();

.##..#..#.###..###..###...##..#..#.####.
#..#.#..#.#..#.#..#.#..#.#..#.#..#....#.
#..#.####.#..#.#..#.#..#.#..#.#..#...#..
####.#..#.###..###..###..####.#..#..#...
#..#.#..#.#....#.#..#....#..#.#..#.#....
#..#.#..#.#....#..#.#....#..#..##..####.
